# Capstone project 1: Predicting the number of visits in US neighborhoods (continued)
# Machine Learning

**Table of Content**
- Introduction
- Machine Learning
- Conclusions

## Introduction
The goal of the machine learning part of the project is to ... 

## Machine Learning

### Step 1: import modules and files

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import operator
import geopandas as gpd
import descartes
import ast
from shapely.geometry import Point , Polygon
from collections import Counter
from sklearn.cluster import OPTICS, cluster_optics_dbscan
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import preprocessing
import time
from math import radians, cos, sin, atan2, sqrt
import random
import hdbscan
%matplotlib inline

/Users/colinechabloz/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/Users/colinechabloz/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
file_path = '/Users/colinechabloz/Desktop/Datasets/Cluster/'
x_cols1_df_n = 'x_cols1_df.csv'

In [3]:
#Load several files from csv
x_cols1_df = pd.read_csv((file_path+x_cols1_df_n), dtype={'census_block_group':str})

In [4]:
x_cols1_df.head()

,Unnamed: 0,census_block_group,raw_visit_count,raw_visitor_count,distance_from_home,related_same_day_brand,related_same_month_brand,top_brands,latitude,longitude,...,vis_home_cbgs_count,vis_home_most_visits_from,vis_home_least_visits_from,avg_vis_home_visitors,visitor_work_cbgs,vis_work_cbgs_count,vis_work_most_visits_from,vis_work_least_visits_from,avg_vis_work_visitors,Cluster
0,0,010059508001,75122.0,18314.0,194724.0,"['chick-fil-a', 'mcdonalds', 'marathonpetroleu...","['walmart', 'mcdonalds', 'dollargeneral', 'chi...","['crossfit', 'healthmart', 'coldwellbanker']",31.920410,-85.175741,...,14,1.005950e+10,1.067030e+10,76.285714,"{'010059501003': 109, '010810407002': 62, '010...",4,1.005950e+10,1.069040e+10,70.000000,2
1,1,010090506011,95649.0,38942.0,120587.0,"['shelloil', 'mcdonalds', 'chick-fil-a', 'chev...","['walmart', 'mcdonalds', 'shelloil', 'chick-fi...",[],33.863884,-86.903514,...,69,1.073011e+10,1.009051e+10,80.623188,"{'010890111001': 271, '010730045001': 269, '01...",41,1.089011e+10,1.077011e+10,93.756098,2
2,2,010479572002,14009.0,3039.0,67774.0,['dollargeneral'],"['walmart', 'dollargeneral', 'mcdonalds', 'che...",['dollargeneral'],32.357337,-87.014004,...,2,1.047957e+10,1.047957e+10,63.500000,{'010479567021': 52},1,1.047957e+10,1.047957e+10,52.000000,0
3,3,010690404001,128169.0,25418.0,42684.0,"['chick-fil-a', 'samsclub', 'dollargeneral', '...","['walmart', 'dollargeneral', 'mcdonalds', 'mar...","['chick-fil-a', 'samsclub', 'olivegarden', 'mc...",31.237327,-85.426331,...,92,1.069040e+10,1.069040e+10,99.000000,"{'010690402024': 313, '010690415004': 203, '01...",23,1.069040e+10,1.031011e+10,108.173913,1
4,4,010730112101,51453.0,9499.0,18878.0,"['chevron', 'daylightdonuts', 'walmart']","['walmart', 'chevron', 'dollargeneral', 'shell...","['chevron', 'crossfit']",33.671800,-86.690610,...,37,1.009051e+10,1.073012e+10,81.513514,"{'010730045001': 140, '010730027001': 123, '01...",10,1.073005e+10,1.073002e+10,86.500000,2


In [5]:
x_cols1_df.shape

(220240, 83)

- filter cluster = 0 
- split train (70%) and test (20%) and validation (10%) to be used at the end to perform new predictions and create plots other than when training/testing the model
- with train set: regression y .. X ... parameters (decide metrics: eg; minimum root mean squared error)
- with test set : predict (same regression)
- measure performance of regression with train and the performance of regression with test 
- compare performances and evaluate the quality of model

In [6]:
#cluster 0
x_cols1_df_0 = x_cols1_df[x_cols1_df['Cluster'] == 0]

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [9]:
y = x_cols1_df_0["raw_visit_count"]

In [10]:
x_cols = np.array(x_cols1_df_0[['12am','1am', '2am', '3am', '4am', '5am', '6am', '7am', '8am', '9am', '10am', '11am', '12pm', '1pm', '2pm', '3pm', '4pm', '5pm', '6pm', '7pm', '8pm', '9pm', '10pm', '11pm', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']])

In [11]:
X = np.asmatrix(x_cols)

In [12]:
X

matrix([[ 807.,  790.,  796., ..., 2076., 1707., 1536.],
        [2459., 2397., 2363., ..., 3712., 3138., 2893.],
        [ 673.,  554.,  484., ..., 2942., 2497., 2257.],
        ...,
        [ 758.,  735.,  735., ...,  952.,  792.,  650.],
        [ 364.,  350.,  358., ...,  375.,  336.,  285.],
        [ 599.,  587.,  563., ...,  943.,  938.,  846.]])

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
reg_all = LinearRegression()
reg_all.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [20]:
y_pred = reg_all.predict(X_test)

In [21]:
reg_all.score(X_test, y_test)

1.0

In [23]:
print("R^2: {}".format(reg_all.score(X_test, y_test)))

R^2: 1.0


In [27]:
rmse = np.sqrt(mean_squared_error(y_test,y_pred))

In [28]:
print("Root Mean Squared Error: {}".format(rmse))

Root Mean Squared Error: 5.968098552999438e-11
